## MLFlow Tester and Uploader
The following will create a local tracking server and will use a remote sftp
for artifact storage
```
sudo apt install python3.10-venv
python3.10 -m venv venv-mlflow
./venv-mlflow/bin/pip install mlflow pysftp
```
Then select the environment as the notebook kernel (may need to refresh the list of available environs)

To start the server...
```
source ./venv-mlflow/bin/activate
mlflow server --backend-store-uri /fantasy-experiments/models/mlflow \
    --default-artifact-root sftp://monkey-king/Users/delano/Archive/fantasy-models \
    --no-serve-artifacts
```

In [ ]:
from glob import glob
import json

import mlflow

EXPERIMENT_NAME = "2023.03 : new-models"
MLFLOW_TRACKING_URI = "http://localhost:5000"

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment is None:
    print("creating experiment")
    experiment_id = mlflow.create_experiment(
        EXPERIMENT_NAME,
    )
    experiment = mlflow.get_experiment(experiment_id)
else:
    print("experiment already exists")
display(experiment)

for filename in glob("MLB*model"):
    print(f"processing {filename}")
    with open(filename, "r") as f_:
        model_json = json.load(f_)
    # print(model_json)

    tags = {
        "name": model_json["name"],
        "sport": model_json["name"].split("-", 1)[0],
        "dt_trained": model_json["dt_trained"],
    }
    params = model_json["parameters"]
    artifacts = [filename, model_json["trained_parameters"]["regressor_path"]]
    performance = {
        k_: v_
        for k_, v_ in model_json["meta_extra"]["performance"].items()
        if k_ != "season"
    }
    params["target"] = ":".join(model_json["training_data_def"]["target"])
    display("tags", tags, f"{performance=}", "artifacts", artifacts, "params", params)

    with mlflow.start_run(experiment_id=experiment.experiment_id) as active_run:
        mlflow.log_metrics(performance)
        mlflow.log_params(params)
        mlflow.set_tags(tags)
        for artifact in artifacts:
            mlflow.log_artifact(artifact)

        display("Active Run", active_run.to_dictionary())

